In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\ABHISHEK SINHA\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [77]:
PINECONE_API_KEY = "40403486-6521-4d7a-8cf7-e77a9840a569"
PINECONE_API_ENV = "us-east1-aws"


In [10]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [12]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [15]:
extracted_data = load_pdf("D:\End_To_End-Medical-Chatbot-Using-Llama2\data")


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

# Use the function to split the extracted data
text_chunks = text_split(extracted_data)
print("Length of my chunks:", len(text_chunks))


Length of my chunks: 7020


In [35]:
#text_chunks

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings

# Download Hugging Face embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()


c:\Users\ABHISHEK SINHA\anaconda3\envs\mchatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\ABHISHEK SINHA\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ABHISHEK SINHA\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to act

In [25]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [26]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [36]:
#query_result

In [75]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [79]:
import os
from pinecone import Pinecone

# Replace with your actual Pinecone API key
PINECONE_API_KEY = "40403486-6521-4d7a-8cf7-e77a9840a569"

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is not set properly.")

# Create an instance of the Pinecone class
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

# Now you can use pinecone_client to perform operations


In [80]:
index_name="medical-chatbot"

In [83]:
import os
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone

# Initialize Pinecone
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # Ensure your API key is set correctly
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is not set properly.")

pinecone.init(api_key=PINECONE_API_KEY)

# Load HuggingFace embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Define your index name
index_name = "medical-chatbot"

# Check if the index exists, create it if not
if index_name not in pinecone.list_indexes():
    print(f"Creating new index: {index_name}")
    pinecone.create_index(index_name, dimension=embeddings.output_dim)  # Specify the dimension

# Create embeddings for each of the text chunks and store them in Pinecone
docsearch = LangchainPinecone.from_texts(
    [t.page_content for t in text_chunks], 
    embeddings, 
    index_name=index_name
)

# Example of performing a similarity search
query = "What are Allergies?"
docs = docsearch.similarity_search(query, k=3)

# Output the results
for doc in docs:
    print("Result:", doc.page_content)


ValueError: PINECONE_API_KEY is not set properly.

In [66]:
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


AttributeError: from_texts is not a top-level attribute of the Pinecone class provided by pinecone's official python package developed at https://github.com/pinecone-io/pinecone-python-client. You may have a name collision with an export from another dependency in your project that wraps Pinecone functionality and exports a similarly named class. Please refer to the following knowledge base article for more information: https://docs.pinecone.io/troubleshooting/pinecone-attribute-errors-with-langchain


In [73]:
query = "What are Allergies"

In [74]:
docs=docsearch.similarity_search(query, k=3)

NameError: name 'docsearch' is not defined

In [ ]:
print("Result", docs)

In [ ]:

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])